In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
from openai import OpenAI
import time
import os
from dataclasses import dataclass, field
from typeguard import typechecked
from typing import Optional
import requests
from tqdm import tqdm
import regex
import glob
import re

In [35]:
projects_df = pd.read_csv(
    r"C:\Users\shaun\OneDrive\Desktop\personal\CS classes\CS classes\COP4934\text2mc\text2mc-dataprocessor\projects.csv"
)

projects_df = projects_df.dropna(subset=["RAW_DOWNLOAD_LINK"])
print(projects_df)

                                                PAGE_URL  \
0      https://www.planetminecraft.com/project/open-l...   
1      https://www.planetminecraft.com/project/shopho...   
2      https://www.planetminecraft.com/project/cosy-t...   
3      https://www.planetminecraft.com/project/crimso...   
5      https://www.planetminecraft.com/project/emeral...   
...                                                  ...   
44245  https://www.planetminecraft.com/project/lava-t...   
44246  https://www.planetminecraft.com/project/tree-h...   
44247  https://www.planetminecraft.com/project/arena-...   
44248  https://www.planetminecraft.com/project/minecr...   
44249  https://www.planetminecraft.com/project/shooti...   

                                            DOWNLOAD_URL  \
0      https://www.mediafire.com/file/221mcei3a2cjtem...   
1      https://www.planetminecraft.com/project/shopho...   
2      https://www.mediafire.com/file/c823rkx5smopz0y...   
3      https://www.planetminecraft.com/

In [36]:
regex_string = r"[^/\\&\?]+\.\w{3,4}(?=([\?&].*$|$))"

def matcher(item):
    match = re.search(regex_string, item)
    if (match):
        return match.group()
    else:
        return None

projects_df["FILENAME"] = projects_df['RAW_DOWNLOAD_LINK'].apply(matcher)
print(projects_df["FILENAME"])

0                           LargeSandDunes.zip
1                 boulevardier-s-shophouse.zip
2               Cosy+Town+Bar+by+ClipperZ0.zip
3        boulevardier-s-crimson-manor-e690.zip
5                                         None
                         ...                  
44245                                     None
44246                                     None
44247                                     None
44248                                     None
44249                                     None
Name: FILENAME, Length: 29044, dtype: object


In [37]:
predownloaded_builds_directory = "D:\\builds\\"
actually_downloaded = glob.glob(os.path.join(predownloaded_builds_directory, "*"))
actually_downloaded = [os.path.basename(path) for path in actually_downloaded]
print(actually_downloaded)

['LargeSandDunes.zip', 'boulevardier-s-shophouse.zip', 'Cosy+Town+Bar+by+ClipperZ0.zip', 'boulevardier-s-crimson-manor-e690.zip', 'emerald-palms.schem', 'Desert+Tavern+2.rar', 'Bayview+3.0.zip', 'Desert_Tavern.rar', 'Village+City+S32+Finale.zip', 'Imperial+Palace+Of+Nosgovia.zip', 'desert-houses-e590.schem', 'Comberton+Village+College%2C+1985%283%29.zip', 'SpawnRailway.zip', 'FarmHouseLandscape.zip', 'creative+build.zip', 'pack.png+castle+%28v+0.2_2%29.rar', 'Cherryvalley.rar', 'Manor+Mansion.zip', 'The+Aftermath+%28world+inside%29.zip', 'vex-decorative-crafting-table-interior-room-worldedit.schem', 'Neon+Fortresses.zip', 'United+States+tjc%286%29.zip', 'Minimalist+Home.zip', 'bridge-e510.zip', 'vex-cozy-wooden-kitchen-room-worldedit.schem', 'abandoned.zip', 'mid-century.zip', 'Angkor+Thom.litematic', 'FNAF+MAP+BY+gZio_.rar', 'Hello+Neighbor+Alpha+2+%5B1.7.10%5D+%282%29.zip', 'boulevardier-s-japanese-chateau-e580.zip', 'Neat+Mountain+with+hole+in+botto.zip', 'Our+World.zip', 'Lake+Cabi

In [38]:
projects_df = projects_df[projects_df["FILENAME"].isin(actually_downloaded)]
projects_df = projects_df.reset_index(drop=True)
projects_df

,PAGE_URL,DOWNLOAD_URL,IMAGE_URL,GPT4_DESCRIPTION,TAGS,RAW_DOWNLOAD_LINK,DOWNLOAD_SIZE,FILENAME
0,https://www.planetminecraft.com/project/open-l...,https://www.mediafire.com/file/221mcei3a2cjtem...,https://static.planetminecraft.com/files/image...,NaN,"['Land Structure', 'Tree', 'Sand', 'Desert', '...",https://download1501.mediafire.com/z9f6f2akrl4...,16875995.0,LargeSandDunes.zip
1,https://www.planetminecraft.com/project/shopho...,https://www.planetminecraft.com/project/shopho...,https://static.planetminecraft.com/files/image...,NaN,"['Fantasy', 'Medieval', 'Big', 'Land Structure...",https://static.planetminecraft.com/files/resou...,753377.0,boulevardier-s-shophouse.zip
2,https://www.planetminecraft.com/project/cosy-t...,https://www.mediafire.com/file/c823rkx5smopz0y...,https://static.planetminecraft.com/files/image...,NaN,"['Land Structure', 'Minecraft', 'Minecraftmap'...",https://download2260.mediafire.com/x9jnt9ohwvm...,3883220.0,Cosy+Town+Bar+by+ClipperZ0.zip
3,https://www.planetminecraft.com/project/crimso...,https://www.planetminecraft.com/project/crimso...,https://static.planetminecraft.com/files/image...,NaN,"['Fantasy', 'Medieval', 'Land Structure', 'Sur...",https://static.planetminecraft.com/files/resou...,985708.0,boulevardier-s-crimson-manor-e690.zip
4,https://www.planetminecraft.com/project/desert...,https://www.mediafire.com/file/35zv12nd816h21p...,https://static.planetminecraft.com/files/image...,NaN,"['Land Structure', 'Minecraft', 'Desert', 'Tav...",https://download2288.mediafire.com/iy9e2qe211b...,1553544.0,Desert+Tavern+2.rar
...,...,...,...,...,...,...,...,...
23108,https://www.planetminecraft.com/project/my-cas...,http://www.mediafire.com/file/glkna1wd9x18uhu/...,NaN,NaN,"['Fantasy', 'Medieval', 'Land Structure', 'Cas...",http://download1075.mediafire.com/lvbqjv7lxuug...,10268176.0,Fear+Castle.zip
23109,https://www.planetminecraft.com/project/portus...,http://www.mediafire.com/?nn8rn2a35uhppp9,NaN,NaN,"['Land Structure', 'Town', 'Village']",http://download1509.mediafire.com/40b8xen3vb9g...,29769866.0,PortusVictoriae_110709.zip
23110,https://www.planetminecraft.com/project/sport-...,http://www.mediafire.com/?ipu9bjbqia573nh,NaN,NaN,"['Land Structure', 'World', 'Sport']",http://download1980.mediafire.com/88lqordoghmg...,16254912.0,Modern-+Castle+age+2.zip
23111,https://www.planetminecraft.com/project/beach-...,http://www.mediafire.com/download.php?w16rbyse...,NaN,NaN,"['Land Structure', 'Minecraft', 'House', 'Mode...",http://download1349.mediafire.com/id11tfqjujzg...,49542265.0,Beach+Town+Project+2.0.zip


In [39]:
allowed_filetypes = [".schematic", ".schem", ".zip", ".rar"]
def filter(row):
    if (row):
        if os.path.splitext(row)[-1] in allowed_filetypes:
            return row
        else:
            print(row)
            return None
    else:
        return None
projects_df["FILENAME"] = projects_df["FILENAME"].apply(filter)
projects_df = projects_df.dropna(subset=["FILENAME"])


easy-drill.nbt
bird-statue.nbt
soviet-store.nbt
old-soviet-school.nbt
brick-soviet-house-square.nbt
brick-soviet-house-1-rectangular.nbt
1920-house.nbt
gryphon.nbt
wall.nbt
watchtower.nbt
clocktower.nbt
darkoakmansion.nbt
farmer.nbt
nitwit.nbt
librarian.nbt
toolsmith.nbt
stonemason.nbt
fletcher.nbt
armorer.nbt
leatherworker.nbt
butcher.nbt
sheperd.nbt
weaponsmith.nbt
cleric.nbt
cartographer.nbt
fisherman.nbt
island-of-love-1-e410.nbt
fd21-3000.nbt
city-hall.nbt
sleepy-shrine.nbt
prostenkiy-domik.nbt
soviet-steam-train-co17.nbt
su-47.nbt
sleepy-island-3.nbt
fox-small-island.nbt
mason-house.nbt
sleepy-island-1.nbt
blue-home-version-2.nbt
victorian-building.nbt
portalfarm.nbt
archeology.nbt
LINK+HOMES.txt
My+100+Days+Survival+Worlds+by+JohnnySau.ZIP
lighthouse-thohul-e870.nbt
FancyHouse.nbt
MediaFire+-+Getting+Started.pdf
Generated+World+-+Shortcut.lnk
dimension.tar
Great+Nordic+Hall+Download+Link.txt
mobfarm.nbt
forge-frontage-vanilla.nbt
terminal3.nbt
medieval-arena-1.nbt
medieval-castl

In [41]:
projects_df
print(len(projects_df))
path = r"C:\Users\shaun\OneDrive\Desktop\personal\CS classes\CS classes\COP4934\text2mc\text2mc-dataprocessor\projects_filtered.csv"
projects_df.to_csv(path)

22901
